In [3]:
#! pip install matplotlib==3.4.3
#! pip install networkx==2.6.3
#! pip install pandas==1.4.1
#! pip install pyspark==3.2.0
#! pip install graphframes==0.6
#! pip install leidenalg==0.9.1

In [1]:
import os
import shutil
from io import StringIO
from datetime import datetime, timedelta

import igraph as ig
import networkx as nx
import pandas as pd
import leidenalg as la
import polars as pl

#from pyspark.sql import functions as sf
#from pyspark import SparkConf
#from pyspark.sql import SparkSession
#from graphframes import GraphFrame

In [4]:
cwd = os.getcwd()
df = pl.read_parquet(cwd + '/HI-Small_Trans_Subset.parquet')

#subset = df.head(300000)

# Specify the output path for the new Parquet file
#output_path = cwd + '/../HI-Small_Trans_Subset.parquet'

# Write the subset DataFrame to a Parquet file
#subset.write_parquet(output_path)

#print(f"Subset saved to {output_path}")

In [5]:
df = df.with_row_count(name="id")
df.head()

C:\Users\blanc\AppData\Local\Temp\ipykernel_31380\3532766406.py:1: DeprecationWarning: `DataFrame.with_row_count` is deprecated. Use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  df = df.with_row_count(name="id")


id,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
u32,str,i64,str,i64,str,f64,str,f64,str,str,i64
0,"""2022/09/01 00:20""",10,"""8000EBD30""",10,"""8000EBD30""",3697.34,"""US Dollar""",3697.34,"""US Dollar""","""Reinvestment""",0
1,"""2022/09/01 00:20""",3208,"""8000F4580""",1,"""8000F5340""",0.01,"""US Dollar""",0.01,"""US Dollar""","""Cheque""",0
2,"""2022/09/01 00:00""",3209,"""8000F4670""",3209,"""8000F4670""",14675.57,"""US Dollar""",14675.57,"""US Dollar""","""Reinvestment""",0
3,"""2022/09/01 00:02""",12,"""8000F5030""",12,"""8000F5030""",2806.97,"""US Dollar""",2806.97,"""US Dollar""","""Reinvestment""",0
4,"""2022/09/01 00:06""",10,"""8000F5200""",10,"""8000F5200""",36682.97,"""US Dollar""",36682.97,"""US Dollar""","""Reinvestment""",0


In [6]:
names = {'id': 'id',
         'Timestamp': 'date',
         'From Bank': 'from_bank',
         'Account': 'from_account',
         'To Bank': 'to_bank',
         'Account.1': 'to_account',
         'Amount Received': 'amount_received',
         'Receiving Currency': 'currency_received',
         'Amount Paid': 'amount_sent',
         'Payment Currency': 'currency_sent',
         'Payment Format': 'payment_method',
         'Is Laundering': 'is_laundering'}

def rename_columns(df, names):
    return df.rename(names)

def timestamp_date(df):
    """Transforms date into datetime format + makes a timestamp column (Unix timestamp)"""
    df = df.with_columns(
        pl.col("date")
        .str.strptime(pl.Datetime, "%Y/%m/%d %H:%M")  # Convert string to Datetime
        .dt.timestamp()  # Convert Datetime to Unix timestamp
        .alias("timestamp"))
  
    df = df.with_columns(
        pl.col("date")
        .str.strptime(pl.Datetime, "%Y/%m/%d %H:%M")
        .dt.date())
    return df

In [7]:
df = rename_columns(df, names)
df = timestamp_date(df)

In [ ]:
#df = df.with_columns((pl.col("from_account") == pl.col("to_account")).alias('same_account'))
#df.group_by("payment_method").agg(
    #((pl.col("same_account").sum() / pl.count()) * 100)
    #.alias("percentage_same_account"))

In [8]:
df.head()

id,date,from_bank,from_account,to_bank,to_account,amount_received,currency_received,amount_sent,currency_sent,payment_method,is_laundering,timestamp
u32,date,i64,str,i64,str,f64,str,f64,str,str,i64,i64
0,2022-09-01,10,"""8000EBD30""",10,"""8000EBD30""",3697.34,"""US Dollar""",3697.34,"""US Dollar""","""Reinvestment""",0,1661991600000000
1,2022-09-01,3208,"""8000F4580""",1,"""8000F5340""",0.01,"""US Dollar""",0.01,"""US Dollar""","""Cheque""",0,1661991600000000
2,2022-09-01,3209,"""8000F4670""",3209,"""8000F4670""",14675.57,"""US Dollar""",14675.57,"""US Dollar""","""Reinvestment""",0,1661990400000000
3,2022-09-01,12,"""8000F5030""",12,"""8000F5030""",2806.97,"""US Dollar""",2806.97,"""US Dollar""","""Reinvestment""",0,1661990520000000
4,2022-09-01,10,"""8000F5200""",10,"""8000F5200""",36682.97,"""US Dollar""",36682.97,"""US Dollar""","""Reinvestment""",0,1661990760000000
